# [IT8701] Introduction to Programming for Data Science: Singapore Labour Market Analysis

# Database Preparation

## Table of Contents
- [Import Libraries & Create MySQL Connection](#1)
- [Create Database Schema](#2)
- [Import Data Tables Into MySQL](#3)
    - [OECD Data](#3_1)
    - [World Bank](#3_2)
    - [Overall Unemployment Rate](#3_2)
    - [Retrenchment](#3_4)
    - [Recruitment/Resignation Rates](#3_5)
    - [Occupational Wages](#3_6)
    - [Weekly Hours](#3_7)
    - [Annual Leave](#3_8)
    - [Other Leave](#3_9)
    - [Flexible Working Arrangements (FWAs)](#3_10)
- [Close MySQL Connection](#4)

# 1. Import Libraries & Create MySQL Connection <a id = '1'></a>

In [1]:
import numpy as np
import pandas as pd
import requests
import mysql.connector
import sys
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format='retina'
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

__Create connection with localhost MySQL server__

In [2]:
# Create connection
con = mysql.connector.connect(user='root', password='1273381F', host='127.0.0.1')

if con.is_connected():
    print('MySQL connection successful!')
else:
    print('Connection failed, lease check MySQL connection.')

cursor = con.cursor()

MySQL connection successful!


# 2. Create Database Schema <a id = '2'></a>

In [3]:
# Create database schema if schema does not yet exist
query = '''
CREATE DATABASE IF NOT EXISTS SGMARKET
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])
finally:
    cursor.close()
    con.close()

# 3. Import Data Tables Into MySQL <a id = '3'></a>

__Re-establish connection with schema for queries to run__

In [4]:
# Create connection
con = mysql.connector.connect(user='root', password='1273381F', host='127.0.0.1', database='SGMARKET')

if con.is_connected():
    print('MySQL connection successful!')
else:
    print('Connection failed, lease check MySQL connection.')

cursor = con.cursor()

MySQL connection successful!


## OECD Data <a id = '3_1'></a>

In [5]:
query = '''
DROP TABLE OECD
'''

cursor.execute(query)
con.commit()

In [6]:
# Read csv
oecd = pd.read_csv('../02_Data/OECD Annual Unemployment Rates (2010-2023).csv')

# Select only relevant columns
oecd_clean = oecd[['TIME_PERIOD', 'REF_AREA', 'Reference area', 'OBS_VALUE']]

# Create table
query = '''
CREATE TABLE IF NOT EXISTS OECD (
    ID INT NOT NULL AUTO_INCREMENT,
    YEAR INT NOT NULL,
    REF_CODE VARCHAR(20) NOT NULL,
    REF_AREA VARCHAR(50) NOT NULL,
    UNEMPLOYMENT_RATE FLOAT(10),
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in oecd_clean.iterrows():
    data = {
        'year': row['TIME_PERIOD'],
        'ref_code': row['REF_AREA'],
        'ref_area': row['Reference area'],
        'unemployment_rate': row['OBS_VALUE']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO OECD (
            YEAR,
            REF_CODE,
            REF_AREA,
            UNEMPLOYMENT_RATE
        )
        VALUES (
            %(year)s,
            %(ref_code)s,
            %(ref_area)s,
            %(unemployment_rate)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM OECD
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,REF_CODE,REF_AREA,UNEMPLOYMENT_RATE
0,1,2010,OECD,OECD,8.53467
1,2,2011,OECD,OECD,8.12688
2,3,2012,OECD,OECD,8.10755
3,4,2013,OECD,OECD,8.01352
4,5,2014,OECD,OECD,7.45507
...,...,...,...,...,...
596,597,2023,JPN,Japan,2.56667
597,598,2010,MEX,Mexico,5.38088
598,599,2010,SVK,Slovak Republic,14.43330
599,600,2010,DNK,Denmark,7.75000


## World Bank <a id = '3_2'></a>

In [7]:
query = '''
DROP TABLE WB_CLASS
'''

cursor.execute(query)
con.commit()

In [8]:
# Read excel
wb_class = pd.read_excel('../02_Data/CLASS.xlsx', sheet_name='List of economies')

# Drop descriptor rows at the bottom of the dataset
wb_class.dropna(subset='Region', inplace=True)
# Replace NaNs with Nones
wb_class.replace(np.nan, None, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS WB_CLASS (
    ID INT NOT NULL AUTO_INCREMENT,
    ECONOMY VARCHAR(150) NOT NULL,
    CODE VARCHAR(50) NOT NULL,
    REGION VARCHAR(150) NOT NULL,
    INCOME_GROUP VARCHAR(150),
    LENDING_CATEGORY VARCHAR(50),
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in wb_class.iterrows():
    data = {
        'economy': row['Economy'],
        'code': row['Code'],
        'region': row['Region'],
        'income_group': row['Income group'],
        'lending_category': row['Lending category']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO WB_CLASS (
            ECONOMY,
            CODE,
            REGION,
            INCOME_GROUP,
            LENDING_CATEGORY
        )
        VALUES (
            %(economy)s,
            %(code)s,
            %(region)s,
            %(income_group)s,
            %(lending_category)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM WB_CLASS
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con).head()

Table creation & data insertion successful!


,ID,ECONOMY,CODE,REGION,INCOME_GROUP,LENDING_CATEGORY
0,1,Afghanistan,AFG,South Asia,Low income,IDA
1,2,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD
2,3,Algeria,DZA,Middle East & North Africa,Upper middle income,IBRD
3,4,American Samoa,ASM,East Asia & Pacific,High income,None
4,5,Andorra,AND,Europe & Central Asia,High income,None


## Define API call function to retreive data.gov.sg datasets

In [9]:
def get_data(resource_id):
    """
    Returns data.gov.sg dataset through use of OpenAPI query.

    Args:
        resource_id (string): Resource ID string pointing to the API endpoint

    Returns:
        pandas dataframe: Dataframe of data.gov.sg dataset
    """
    # Define key arguments for data retrieval and formatting
    base_url = 'https://data.gov.sg/api/action/datastore_search'
    offset = 0
    df = []

    while True:
        # Set url and parameters for API endpoint
        parameters = {
            'resource_id': resource_id,
            'offset': offset # define offset
        }
        # Retrieve data
        response = requests.get(base_url, params=parameters)
        # Check if request is successful
        if response.status_code == 200:
            # Retrieve json text
            response_dict = response.json()
            # Check if at least 1 record
            if len(response_dict['result']['records']) > 0:
                # Normalize and append records to list, df
                df_temp = pd.json_normalize(response_dict['result']['records'])
                df.append(df_temp)
                # Continue the loop and increase offset to retrieve the next 100 records
                offset += 100
            # Repeat retrieval of every 100 records until the request returns 0 records - break loop
            else:
                break
        # If request not successful, print error message with status code
        else:
            print(f'The request failed and returned status code: {response.status_code}')
            
    # Concatenate list of dataframes
    df = pd.concat(df, ignore_index=True)
    # Return df
    return df

## Overall Unemployment Rate <a id = '3_3'></a>

In [10]:
query = '''
DROP TABLE SG_UNEMPLOYMENT
'''

cursor.execute(query)
con.commit()

In [11]:
# Retrieve data
sg_unemployment = get_data('d_e3598914c86699a9a36e68190f78c59a')

# Create table
query = '''
CREATE TABLE IF NOT EXISTS SG_UNEMPLOYMENT (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    RESIDENTIAL_STATUS VARCHAR(20) NOT NULL,
    UNEMPLOYMENT_RATE FLOAT(10) NOT NULL,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in sg_unemployment.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'residential_status': row['residential_status'],
        'unemployment_rate': row['unemployment_rate']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO SG_UNEMPLOYMENT (
            ID,
            YEAR,
            RESIDENTIAL_STATUS,
            UNEMPLOYMENT_RATE
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(residential_status)s,
            %(unemployment_rate)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM SG_UNEMPLOYMENT
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con).head()

Table creation & data insertion successful!


,ID,YEAR,RESIDENTIAL_STATUS,UNEMPLOYMENT_RATE
0,1,1992,overall,1.8
1,2,1993,overall,1.7
2,3,1994,overall,1.7
3,4,1995,overall,1.8
4,5,1996,overall,1.7


## Retrenchment <a id = '3_4'></a>

In [12]:
query = '''
DROP TABLE RETRENCHMENT
'''

cursor.execute(query)
con.commit()

In [13]:
# Retrieve data
retrenchment = get_data('d_c794e2ad25c1d3bb7965d13dffd0b411')

# Data cleaning for importing into MySQL
retrenchment.replace('-', None, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS RETRENCHMENT (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    INDUSTRY2 VARCHAR(100) NOT NULL,
    INDUSTRY3 VARCHAR(100) NOT NULL,
    RETRENCH INT,
    RETRENCH_PERM INT,
    RETRENCH_CONTRACT INT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in retrenchment.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'industry1': row['industry1'],
        'industry2': row['industry2'],
        'industry3': row['industry3'],
        'retrench': row['retrench'],
        'retrench_perm': row['retrench_permanent'],
        'retrench_contract': row['retrench_term_contract']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO RETRENCHMENT (
            ID,
            YEAR,
            INDUSTRY1,
            INDUSTRY2,
            INDUSTRY3,
            RETRENCH,
            RETRENCH_PERM,
            RETRENCH_CONTRACT
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(industry1)s,
            %(industry2)s,
            %(industry3)s,
            %(retrench)s,
            %(retrench_perm)s,
            %(retrench_contract)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM RETRENCHMENT
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,INDUSTRY1,INDUSTRY2,INDUSTRY3,RETRENCH,RETRENCH_PERM,RETRENCH_CONTRACT
0,1,1998,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",230.0,230.0,NaN
1,2,1998,manufacturing,textile and wearing apparel,textile and wearing apparel,310.0,310.0,10.0
2,3,1998,manufacturing,paper products and publishing,paper products and publishing,510.0,510.0,NaN
3,4,1998,manufacturing,petroleum and chemical products,petroleum and chemical products,500.0,490.0,10.0
4,5,1998,manufacturing,rubber and plastic products,rubber and plastic products,1100.0,1060.0,50.0
...,...,...,...,...,...,...,...,...
821,822,2023,services,"community, social and personal services",public administration and education,60.0,50.0,NaN
822,823,2023,services,"community, social and personal services",health and social services,80.0,60.0,10.0
823,824,2023,services,"community, social and personal services","arts, entertainment and recreation",30.0,30.0,10.0
824,825,2023,services,"community, social and personal services","other community, social and personal services",120.0,120.0,NaN


## Recruitment/Resignation Rates <a id = '3_5'></a>

In [14]:
query = '''
DROP TABLE TURNOVER
'''

cursor.execute(query)
con.commit()

In [15]:
# Retrieve data
turnover = get_data('d_e6aeec36ef233579b42fa4a53d6b68cb')

# Data cleaning for importing into MySQL
turnover.replace('-', None, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS TURNOVER (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    INDUSTRY2 VARCHAR(100) NOT NULL,
    INDUSTRY3 VARCHAR(100) NOT NULL,
    OCCUPATION VARCHAR(100) NOT NULL,
    RECRUITMENT FLOAT,
    RESIGNATION FLOAT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in turnover.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'industry1': row['industry1'],
        'industry2': row['industry2'],
        'industry3': row['industry3'],
        'occupation': row['occupation1'],
        'recruitment': row['recruitment_rate'],
        'resignation': row['resignation_rate']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO TURNOVER (
            ID,
            YEAR,
            INDUSTRY1,
            INDUSTRY2,
            INDUSTRY3,
            OCCUPATION,
            RECRUITMENT,
            RESIGNATION
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(industry1)s,
            %(industry2)s,
            %(industry3)s,
            %(occupation)s,
            %(recruitment)s,
            %(resignation)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM TURNOVER
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,INDUSTRY1,INDUSTRY2,INDUSTRY3,OCCUPATION,RECRUITMENT,RESIGNATION
0,1,1998,total,total,total,total,2.3,2.1
1,2,1998,total,total,total,"professional, managers, executive and technicians",1.8,1.4
2,3,1998,total,total,total,"clerical, sales and services workers",2.9,2.9
3,4,1998,total,total,total,"production and transport operators, cleaners a...",2.3,2.0
4,5,1998,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",total,3.5,3.1
...,...,...,...,...,...,...,...,...
3403,3404,2023,services,"community, social and personal services","other community, social and personal services","production and transport operators, cleaners a...",1.8,1.3
3404,3405,2023,others,others,others,total,1.5,1.0
3405,3406,2023,others,others,others,"professional, managers, executive and technicians",1.4,0.7
3406,3407,2023,others,others,others,"clerical, sales and services workers",1.9,1.1


## Occupational Wages <a id = '3_6'></a>

In [16]:
query = '''
DROP TABLE WAGES
'''

cursor.execute(query)
con.commit()

In [17]:
# Retrieve data
wage = get_data('d_43a37d383fe7b33368f1a3465467571d')

# Data cleaning for importing into MySQL
wage.replace('-', None, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS WAGES (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    OCCUPATION VARCHAR(100) NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    OCCUPATION_DESC VARCHAR(999) NOT NULL,
    MTHLY_GROSS INT,
    MTHLY_BASIC INT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in wage.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'occupation': row['major_occ'],
        'industry1': row['ind1'],
        'occupation_desc': row['occ_desc'],
        'mthly_gross': row['mthly_gross_wage_50_pctile'],
        'mthly_basic': row['mthly_basic_wage_50_pctile']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO WAGES (
            ID,
            YEAR,
            OCCUPATION,
            INDUSTRY1,
            OCCUPATION_DESC,
            MTHLY_GROSS,
            MTHLY_BASIC
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(occupation)s,
            %(industry1)s,
            %(occupation_desc)s,
            %(mthly_gross)s,
            %(mthly_basic)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM WAGES
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,OCCUPATION,INDUSTRY1,OCCUPATION_DESC,MTHLY_GROSS,MTHLY_BASIC
0,1,2022,"professionals, managers & executives",manufacturing,industrial and production engineer,5426,4824
1,2,2022,"professionals, managers & executives",manufacturing,transport and vehicle engineer,5223,4500
2,3,2022,"professionals, managers & executives",manufacturing,manufacturing manager,7968,7532
3,4,2022,"professionals, managers & executives","food, beverages & tobacco",manufacturing manager,5000,4500
4,5,2022,"professionals, managers & executives","food, beverages & tobacco",sales manager,7315,3600
...,...,...,...,...,...,...,...
355,356,2022,"production & transport operators, cleaners & l...",health & social services,attendant,2171,2171
356,357,2022,"production & transport operators, cleaners & l...",health & social services,ambulance driver,2702,1700
357,358,2022,"production & transport operators, cleaners & l...","arts, entertainment, recreation and other serv...","office, commercial and industrial establishmen...",1927,1600
358,359,2022,"production & transport operators, cleaners & l...","arts, entertainment, recreation and other serv...",chauffeur,2519,2493


## Weekly Hours <a id = '3_7'></a>

In [18]:
query = '''
DROP TABLE WEEKLY_HOURS
'''

cursor.execute(query)
con.commit()

In [19]:
# Retrieve data
weekly_hours = get_data('d_fad3e6e2455bf38a1dcf1b11f436cd88')

# Data cleaning for importing into MySQL
weekly_hours.replace('-', None, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS WEEKLY_HOURS (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    INDUSTRY2 VARCHAR(100) NOT NULL,
    INDUSTRY3 VARCHAR(100) NOT NULL,
    EMPLOYMENT_NATURE VARCHAR(20) NOT NULL,
    TOTAL_PAID_HOURS FLOAT,
    STANDARD_HOURS FLOAT,
    OT_ALL_EMPLOYEES FLOAT,
    OT_OT_EMPLOYEES FLOAT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in weekly_hours.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'industry1': row['industry1'],
        'industry2': row['industry2'],
        'industry3': row['industry3'],
        'nature': row['nature_of_employment'],
        'total_hours': row['total_paid_hours'],
        'standard_hours': row['standard_hours'],
        'OT_all': row['overtime_of_all_employees'],
        'OT_OT_only': row['overtime_of_employees_who_worked_overtime']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO WEEKLY_HOURS (
            ID,
            YEAR,
            INDUSTRY1,
            INDUSTRY2,
            INDUSTRY3,
            EMPLOYMENT_NATURE,
            TOTAL_PAID_HOURS,
            STANDARD_HOURS,
            OT_ALL_EMPLOYEES,
            OT_OT_EMPLOYEES
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(industry1)s,
            %(industry2)s,
            %(industry3)s,
            %(nature)s,
            %(total_hours)s,
            %(standard_hours)s,
            %(OT_all)s,
            %(OT_OT_only)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM WEEKLY_HOURS
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,INDUSTRY1,INDUSTRY2,INDUSTRY3,EMPLOYMENT_NATURE,TOTAL_PAID_HOURS,STANDARD_HOURS,OT_ALL_EMPLOYEES,OT_OT_EMPLOYEES
0,1,1998,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",all,44.4,41.7,2.6,7.6
1,2,1998,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",full-time,46.3,43.5,2.8,7.6
2,3,1998,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",part-time,23.3,22.8,0.6,8.1
3,4,1998,manufacturing,textile and wearing apparel,textile and wearing apparel,all,49.5,43.4,6.1,10.2
4,5,1998,manufacturing,textile and wearing apparel,textile and wearing apparel,full-time,50.2,43.9,6.2,10.2
...,...,...,...,...,...,...,...,...,...,...
2473,2474,2023,services,"community, social and personal services","other community, social and personal services",full-time,43.1,42.4,0.7,6.0
2474,2475,2023,services,"community, social and personal services","other community, social and personal services",part-time,22.0,21.9,0.1,3.7
2475,2476,2023,others,others,others,all,44.4,42.1,2.3,10.0
2476,2477,2023,others,others,others,full-time,44.8,42.5,2.3,10.0


## Annual Leave <a id = '3_8'></a>

In [20]:
query = '''
DROP TABLE ANNUAL_LEAVE
'''

cursor.execute(query)
con.commit()

In [21]:
# Retrieve data
annual_leave = get_data('d_a1a97c36b2299a0805d8eabd4774da72')

# Data cleaning for importing into MySQL
annual_leave.replace({'-': None, 'na': None, 's': None}, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS ANNUAL_LEAVE (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    INDUSTRY2 VARCHAR(100) NOT NULL,
    INDUSTRY3 VARCHAR(100) NOT NULL,
    ANNUAL_LEAVE VARCHAR(100),
    DISTRIBUTION FLOAT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in annual_leave.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'industry1': row['industry1'],
        'industry2': row['industry2'],
        'industry3': row['industry3'],
        'annual_leave': row['annual_leave_entitlement'],
        'distribution': row['distribution']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO ANNUAL_LEAVE (
            ID,
            YEAR,
            INDUSTRY1,
            INDUSTRY2,
            INDUSTRY3,
            ANNUAL_LEAVE,
            DISTRIBUTION
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(industry1)s,
            %(industry2)s,
            %(industry3)s,
            %(annual_leave)s,
            %(distribution)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM ANNUAL_LEAVE
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,INDUSTRY1,INDUSTRY2,INDUSTRY3,ANNUAL_LEAVE,DISTRIBUTION
0,1,2012,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",14 days & below,77.2
1,2,2012,manufacturing,paper/ rubber/ plastic products and printing,paper/ rubber/ plastic products and printing,14 days & below,65.7
2,3,2012,manufacturing,"petroleum, chemical and pharmaceutical products","petroleum, chemical and pharmaceutical products",14 days & below,21.0
3,4,2012,manufacturing,"fabricated metal products, machinery and equip...","fabricated metal products, machinery and equip...",14 days & below,67.9
4,5,2012,manufacturing,"electronic, computer and optical products","electronic, computer and optical products",14 days & below,48.9
...,...,...,...,...,...,...,...
535,536,2022,services,administrative and support services,cleaning and landscaping,over 21 days,0.1
536,537,2022,services,administrative and support services,other administrative and support services,over 21 days,3.6
537,538,2022,services,"community, social and personal services",education,over 21 days,34.7
538,539,2022,services,"community, social and personal services",health and social services,over 21 days,26.6


## Other Leave <a id = '3_9'></a>

In [22]:
query = '''
DROP TABLE OTHER_LEAVE
'''

cursor.execute(query)
con.commit()

In [23]:
# Retrieve data
other_leave = get_data('d_c8db149104046725174a56b9d3e79926')

# Data cleaning for importing into MySQL
other_leave.replace({'-': None, 'na': None, 's': None}, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS OTHER_LEAVE (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    INDUSTRY2 VARCHAR(100) NOT NULL,
    INDUSTRY3 VARCHAR(100) NOT NULL,
    COMPASSIONATE_LEAVE FLOAT,
    MARRIAGE_LEAVE FLOAT,
    UNPAID_LEAVE_MORETHAN_1MTH FLOAT,
    UNPAID_LEAVE_LESSTHAN_1MTH FLOAT,
    STUDY_LEAVE FLOAT,
    CHILD_SICK_LEAVE FLOAT,
    PARENT_CARE_LEAVE FLOAT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in other_leave.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'industry1': row['industry1'],
        'industry2': row['industry2'],
        'industry3': row['industry3'],
        'compassionate_leave': row['compassionate_leave'],
        'marriage_leave': row['marriage_leave'],
        'unpaid_leave_morethan_1mth': row['unpaid_leave_morethan1mth'],
        'unpaid_leave_lessthan_1mth': row['unpaid_leave_1mthorless'],
        'study_leave': row['study_examination_leave'],
        'child_sick_leave': row['child_sick_leave'],
        'parent_care_leave': row['parent_care_leave']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO other_LEAVE (
            ID,
            YEAR,
            INDUSTRY1,
            INDUSTRY2,
            INDUSTRY3,
            COMPASSIONATE_LEAVE,
            MARRIAGE_LEAVE,
            UNPAID_LEAVE_MORETHAN_1MTH,
            UNPAID_LEAVE_LESSTHAN_1MTH,
            STUDY_LEAVE,
            CHILD_SICK_LEAVE,
            PARENT_CARE_LEAVE
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(industry1)s,
            %(industry2)s,
            %(industry3)s,
            %(compassionate_leave)s,
            %(marriage_leave)s,
            %(unpaid_leave_morethan_1mth)s,
            %(unpaid_leave_lessthan_1mth)s,
            %(study_leave)s,
            %(child_sick_leave)s,
            %(parent_care_leave)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM WFH
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,INDUSTRY1,INDUSTRY2,INDUSTRY3,AT_LEAST_ONE,PART_TIME,FLEX_TIME,STAGGERED_HOURS,TELE_WORK,HOME_WORK,JOB_SHARING,COMPRESSED_WEEK
0,1,2011,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",29.8,20.7,5.9,5.9,NaN,NaN,NaN,NaN
1,2,2011,manufacturing,paper/ rubber/ plastic products and printing,paper/ rubber/ plastic products and printing,22.8,13.3,7.1,2.1,2.1,NaN,NaN,NaN
2,3,2011,manufacturing,"petroleum, chemical and pharmaceutical products","petroleum, chemical and pharmaceutical products",31.9,12.3,16.0,11.7,4.9,1.2,2.5,NaN
3,4,2011,manufacturing,fabricated metal products and machinery,fabricated metal products and machinery,27.6,20.3,6.8,1.5,2.5,0.7,1.2,NaN
4,5,2011,manufacturing,electronic and computer products,electronic and computer products,27.3,14.0,8.4,11.2,2.8,0.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,2022,services,administrative and support services,cleaning and landscaping,63.0,50.3,25.3,23.3,14.8,7.5,12.5,14.0
356,357,2022,services,administrative and support services,other administrative and support services,71.5,30.2,37.1,43.6,39.2,4.1,16.2,19.6
357,358,2022,services,"community, social and personal services",education,96.0,73.2,46.0,69.2,51.5,18.7,34.8,30.3
358,359,2022,services,"community, social and personal services",health and social services,82.2,64.0,43.8,47.3,36.3,12.0,17.1,17.1


## Flexible Working Arrangements (FWAs) <a id = '3_10'></a>

In [24]:
query = '''
DROP TABLE WFH
'''

cursor.execute(query)
con.commit()

In [25]:
# Retrieve data
wfh = get_data('d_9190c1b4524b6361ce6afdffd39fa94e')

# Data cleaning for importing into MySQL
wfh.replace({'-': None, 'na': None, 's': None}, inplace=True)

# Create table
query = '''
CREATE TABLE IF NOT EXISTS WFH (
    ID INT NOT NULL,
    YEAR INT NOT NULL,
    INDUSTRY1 VARCHAR(100) NOT NULL,
    INDUSTRY2 VARCHAR(100) NOT NULL,
    INDUSTRY3 VARCHAR(100) NOT NULL,
    AT_LEAST_ONE FLOAT,
    PART_TIME FLOAT,
    FLEX_TIME FLOAT,
    STAGGERED_HOURS FLOAT,
    TELE_WORK FLOAT,
    HOME_WORK FLOAT,
    JOB_SHARING FLOAT,
    COMPRESSED_WEEK FLOAT,
    PRIMARY KEY (ID)
)
'''

try:
    cursor.execute(query)
    con.commit()
except:
    print('Unexpected error:', sys.exc_info()[0])
    print(sys.exc_info()[1])

# Insert data into table
for index, row in wfh.iterrows():
    data = {
        'id': row['_id'],
        'year': row['year'],
        'industry1': row['industry1'],
        'industry2': row['industry2'],
        'industry3': row['industry3'],
        'at_least_one': row['at_least_one'],
        'part_time': row['part-time'],
        'flex_time': row['flexitime'],
        'staggered_hours': row['staggered_hours'],
        'tele_work': row['tele-working'],
        'home_work': row['homeworking'],
        'job_sharing': row['job-sharing'],
        'compressed_week': row['compressed_work_week']
    }

    # %(key)s: This syntax is used for string formatting where key is a placeholder that will be replaced by the value associated with key in a dictionary.
    query = '''
        INSERT INTO WFH (
            ID,
            YEAR,
            INDUSTRY1,
            INDUSTRY2,
            INDUSTRY3,
            AT_LEAST_ONE,
            PART_TIME,
            FLEX_TIME,
            STAGGERED_HOURS,
            TELE_WORK,
            HOME_WORK,
            JOB_SHARING,
            COMPRESSED_WEEK
        )
        VALUES (
            %(id)s,
            %(year)s,
            %(industry1)s,
            %(industry2)s,
            %(industry3)s,
            %(at_least_one)s,
            %(part_time)s,
            %(flex_time)s,
            %(staggered_hours)s,
            %(tele_work)s,
            %(home_work)s,
            %(job_sharing)s,
            %(compressed_week)s
        )
    '''
    cursor.execute(query, data)
    con.commit()

# View data
query = '''
SELECT * FROM WFH
'''

if len(pd.read_sql(query, con)) != 0:
    print('Table creation & data insertion successful!')
else:
    print('Issue with table creation & data insertion process, please check.')

pd.read_sql(query, con)

Table creation & data insertion successful!


,ID,YEAR,INDUSTRY1,INDUSTRY2,INDUSTRY3,AT_LEAST_ONE,PART_TIME,FLEX_TIME,STAGGERED_HOURS,TELE_WORK,HOME_WORK,JOB_SHARING,COMPRESSED_WEEK
0,1,2011,manufacturing,"food, beverages and tobacco","food, beverages and tobacco",29.8,20.7,5.9,5.9,NaN,NaN,NaN,NaN
1,2,2011,manufacturing,paper/ rubber/ plastic products and printing,paper/ rubber/ plastic products and printing,22.8,13.3,7.1,2.1,2.1,NaN,NaN,NaN
2,3,2011,manufacturing,"petroleum, chemical and pharmaceutical products","petroleum, chemical and pharmaceutical products",31.9,12.3,16.0,11.7,4.9,1.2,2.5,NaN
3,4,2011,manufacturing,fabricated metal products and machinery,fabricated metal products and machinery,27.6,20.3,6.8,1.5,2.5,0.7,1.2,NaN
4,5,2011,manufacturing,electronic and computer products,electronic and computer products,27.3,14.0,8.4,11.2,2.8,0.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,356,2022,services,administrative and support services,cleaning and landscaping,63.0,50.3,25.3,23.3,14.8,7.5,12.5,14.0
356,357,2022,services,administrative and support services,other administrative and support services,71.5,30.2,37.1,43.6,39.2,4.1,16.2,19.6
357,358,2022,services,"community, social and personal services",education,96.0,73.2,46.0,69.2,51.5,18.7,34.8,30.3
358,359,2022,services,"community, social and personal services",health and social services,82.2,64.0,43.8,47.3,36.3,12.0,17.1,17.1


## Close MySQL Connection <a id = '4'></a>

In [26]:
cursor.close()
con.close()